# Organizing and Visualizing Datas -3. Number of Coffee Shops Near Subway Stations in Seoul

In [1]:
# Import all the necessary liabraries
import numpy as np 
import pandas as pd 
pd.set_option('display.max_rows', None)
import json 
from pandas.io.json import json_normalize 

import requests 

import branca
import folium 

print('Libraries imported.')

Libraries imported.


## 1. Organize the data : Getting coffee shop data using Foursquare API

In [2]:
#Load Subway+Price data, add cafe count
%store -r data_subprice
df_subprice = data_subprice
df_subprice['Cafe_Count'] = 0
df_subprice.head()

,Sta_ID,Alight_Num,Sta_Name_,X_Coor,Y_Coor,Addr,Fee_Space,Cafe_Count
0,150,3110466,서울,37.554648,126.972559,서울특별시 용산구 동자동,947.794016,0
1,151,1759078,시청,37.564718,126.977108,서울특별시 중구 태평로2가,673.177836,0
2,152,1361926,종각,37.570161,126.982923,서울특별시 종로구 종로1가,1188.834528,0
3,153,1979876,종로3가,37.571607,126.991806,서울특별시 종로구 묘동,852.820360,0
4,154,874720,종로5가,37.570926,127.001849,서울특별시 종로구 종로5가,895.911333,0


In [3]:
#Initialize variables for Foursquare API
CLIENT_ID = #Removed Foursquare credentials
CLIENT_SECRET = #Removed Foursquare credentials
VERSION = '20191212'
search_query = '4bf58dd8d48988d1e0931735,4bf58dd8d48988d16d941735' #Coffee shops, Cafes
radius=100 #Venues within 100 meter radius of a coordinate
LIMIT=50 # Set Max return to 50(The Maximun return of the API). The highest returned count is 49, so no loss in data.

In [4]:
#Request Data
for i in range(0, df_subprice.shape[0]):
    latitude = df_subprice.iloc[i,3]
    longitude = df_subprice.iloc[i,4]
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius,LIMIT)
    results = requests.get(url).json()
    try:
        venues = results['response']['venues']
        dataframe = json_normalize(venues)
        df_subprice.loc[i,'Cafe_Count'] = dataframe.shape[0] #Get the number of rows from returned data to count the coffee shops
    except:
        df_subprice.loc[i,'Cafe_Count'] = 0

In [5]:
print(df_subprice.shape)
df_subprice.head()

(279, 8)


,Sta_ID,Alight_Num,Sta_Name_,X_Coor,Y_Coor,Addr,Fee_Space,Cafe_Count
0,150,3110466,서울,37.554648,126.972559,서울특별시 용산구 동자동,947.794016,19
1,151,1759078,시청,37.564718,126.977108,서울특별시 중구 태평로2가,673.177836,21
2,152,1361926,종각,37.570161,126.982923,서울특별시 종로구 종로1가,1188.834528,38
3,153,1979876,종로3가,37.571607,126.991806,서울특별시 종로구 묘동,852.820360,20
4,154,874720,종로5가,37.570926,127.001849,서울특별시 종로구 종로5가,895.911333,28


In [6]:
#Store Data
data_final = df_subprice
%store data_final
del data_final

Stored 'data_final' (DataFrame)


## 2. Visualize the data

In [7]:
#Create scale for color
threshold_scale = np.linspace(df_subprice['Cafe_Count'].min(),
                              df_subprice['Cafe_Count'].max(),
                              6, dtype=int)
threshold_scale = threshold_scale.tolist()
threshold_scale

[0, 9, 19, 29, 39, 49]

In [8]:
#Create Map
map_seoul = folium.Map(location=[37.5666791, 126.9782914],zoom_start=12,tiles='Stamen Toner')

for lat, lng, label, pop in zip(df_subprice['X_Coor'], df_subprice['Y_Coor'], df_subprice['Sta_Name_'], df_subprice['Cafe_Count']):
    label = folium.Popup(label, parse_html=True)
    if pop in range(threshold_scale[0], threshold_scale[1]):
        color = '#ff0000'
    elif pop in range(threshold_scale[1], threshold_scale[2]):
        color = '#f6ff00'
    elif pop in range(threshold_scale[2], threshold_scale[3]):
        color = '#00ff08'
    elif pop in range(threshold_scale[3], threshold_scale[4]):
        color = '#0d00ff'
    elif pop in range(threshold_scale[4], threshold_scale[5]+1):
        color = '#ff00f7'
        
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=color,
        fill=True,
        fill_color='#000000',
        fill_opacity=1,
        parse_html=False).add_to(map_seoul)  

In [9]:
#For Map Legend
legend_html = '''
{% macro html(this, kwargs) %}
<div style="
    position: fixed; 
    top: 0px;
    right: -100px;
    width: 350px;
    height: 80px;
    z-index:9999;
    font-size:14px;
    ">
    <p></a>&emsp;Coffee Shops Within 100m Of Stations</p>
    <p><a style="color:#ff0000;font-size:150%;margin-left:20px;">●</a>&emsp;Least Coffee Shops</p>
    <p><a style="color:#f6ff00;font-size:150%;margin-left:20px;">●</a>&emsp;Less Coffee Shops</p>
    <p><a style="color:#00ff08;font-size:150%;margin-left:20px;">●</a>&emsp;Medium Coffee Shops</p>
    <p><a style="color:#0d00ff;font-size:150%;margin-left:20px;">●</a>&emsp;More Coffee Shops</p>
    <p><a style="color:#ff00f7;font-size:150%;margin-left:20px;">●</a>&emsp;Most Coffee Shops</p>
</div>
<div style="
    position: fixed; 
    top: 0px;
    right: 0px;
    width: 250px;
    height: 220px; 
    z-index:9998;
    font-size:14px;
    background-color: #E0FFFF;
    opacity: 0.8;
    ">
</div>
{% endmacro %}
'''

legend = branca.element.MacroElement()
legend._template = branca.element.Template(legend_html)

map_seoul.get_root().add_child(legend)